## Imported Libraries 

In [2]:
import os
import sys
import pandas as pd 
import numpy as np 
import networkx as nx 
from pyvis.network import Network

## Load Files

In [3]:
BASE_DIR = r"S:\WP Personnel\A - Workforce\Data science student\Desys Corner"
STAFF_FILE = os.path.join(BASE_DIR, "Staff_List_New.csv")
HIRES_FILE = os.path.join(BASE_DIR, "NHS_Hires_Detail.csv")
LEAVERS_FILE = os.path.join(BASE_DIR, "NHS_Leavers_Detail.csv")

# Verification of each CSV
for file_path in (STAFF_FILE, HIRES_FILE, LEAVERS_FILE):
    if not os.path.isfile(file_path):
        print(f":ERROR: Missing file:\n {file_path}\n"
              f"Please confirm that this exact path and filename are correct.")
        sys.exit(1)

## Clean Data 

In [4]:
df_staff = pd.read_csv(
    STAFF_FILE,
    low_memory=False
)

df_staff.columns = (
    df_staff.columns.str.strip().str.lower().str.replace(r"[() ]+", "_", regex=True)
)


df_staff['month'] = pd.to_datetime(df_staff['month'], format="%Y-%m-%d", errors='coerce')

In [6]:

for col in ('service_group', 'staff_group', 'department', 'organisation'):
    if col in df_staff.columns:
        df_staff[col] = df_staff[col].astype(str).str.title().str.strip()


df_staff = df_staff.sort_values(['employee_number', 'month']).reset_index(drop=True)

In [8]:

df_hires = pd.read_csv(
    HIRES_FILE,
    low_memory=False
)

df_hires.columns = (
     df_hires.columns.str.strip().str.lower().str.replace(r"[() ]+", "_", regex=True)
)


df_leavers = pd.read_csv(
    LEAVERS_FILE,
    low_memory=False
)

df_leavers.columns = (
     df_leavers.columns.str.strip().str.lower().str.replace(r"[() ]+", "_", regex=True)
)

## Exclude External Hires/Leavers

In [ ]:
external_ids = set(df_hires['employee_number']).union(df_leavers['employee_number'])
df_staff_int = df_staff[~df_staff['employee_number'].isin(external_ids)].copy()
if df_staff_int.empty:
    print("No internal staff records remain.")
    sys.exit(0)

## Design NetworkX Graph

In [41]:
G = nx.DiGraph() 
# Add hire and leave nodes 
G.add_node("HIRE", label="HIRE", title="External Hire", color="green", size=20)
G.add_node("LEAVE", label="LEAVE", title="External Leave", color="red", size=20)
added_nodes = set()

#Loop through employees history month by month then create nodes
grouped = df_staff_int.groupby('employee_number')
for emp_id, df_emp in grouped:
    df_emp = df_emp.sort_values('month').reset_index(drop=True)
    prev_assign = None
    for idx, row in df_emp.iterrows():
        assign_id = str(row['assignment_number'])
        #Node attributes 
        month_val = row['month']
        if pd.isna(month_val):
            month_str = ""
        else:
            month_str = month_val.strftime('%Y-%m')
        title_html = (
            f"Emp: {emp_id}<br>Dept: {row.get('department','')}<br>"
            f"Svc: {row.get('service_group','')}<br>Org: {row.get('organisation','')}<br>"
            f"Month: {month_str}"
        )
    node_attrs = {
        'label': assign_id,
        'title': title_html,
        'department': row.get('department',''),
        'service_group': row.get('service_group',''),
        'organisation': row.get('organisation',''),
        'month_str': month_str,
        'color': "blue",
        'size': 15
    }
    if assign_id not in added_nodes:
        G.add_node(assign_id, **node_attrs)
        added_nodes.add(assign_id)
    if prev_assign is None:
        G.add_edge("HIRE", assign_id, employee=emp_id)
    else:
        G.add_edge(prev_assign, assign_id, employee=emp_id)
    prev_assign = assign_id

G.add_edge(prev_assign, "LEAVE", employee=emp_id)


## Visualise W/ Pyvis

In [42]:
net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
# Loaded NetworkX graph into Pyvis
net.from_nx(G)

#Enable physics for interactive layout 
net.toggle_physics(True)

#Save interactive HTML 
output_file= os.path.join(BASE_DIR, "internal_moves_network.html")
net.write_html(output_file)
print(f"Network visualisation saved to {output_file}")

Network visualisation saved to S:\WP Personnel\A - Workforce\Data science student\Desys Corner\internal_moves_network.html
